# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://www.jaivikhimalay.com")
ed.links

['/cart',
 '#page',
 '/',
 '#services',
 '#about',
 '#contact',
 '/',
 '#services',
 '#about',
 '#contact',
 '#services',
 '#about',
 '#contact',
 '#services',
 'http://squarespace.com']

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.jaivikhimalay.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/cart
#page
/
#services
#about
#contact
/
#services
#about
#contact
#services
#about
#contact
#services
http://squarespace.com


In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
anthropic = Website("https://anthropic.com")
anthropic.links

['/',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/3-5-models-and-computer-use',
 '/claude/sonnet',
 '/claude/haiku',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosure-policy',
 'https://trust.anthropic.com/']

In [11]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://anthropic.com/team'},
  {'type': 'research page', 'url': 'https://anthropic.com/research'},
  {'type': 'news page', 'url': 'https://anthropic.com/news'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Learn more
Announcements
Introducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku
Oct 22, 2024
Model updates
3.5 Sonnet
3.5 Haiku
Our Work
Product
Claude for Enterprise
Sep 4, 2024
Alignment
·
Research
Constitutional A

In [16]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://www.anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}


"You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nStart using Claude to drive efficiency and create new revenue streams.\nLearn more\nAnnouncements\nIntroducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku\nOct 22, 2024\nModel updates\n3.5 Sonnet\n3.5 Haiku\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 2024\nAlignment\n·\nResearch\nConstitutional AI: Harmlessness from AI Feedback\nDec 15, 2022\nAnnouncements\nCore Views on AI Safety: When, Why, What, and How\nMar 8, 2023\nWo

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}


# Anthropic Company Brochure

## Overview
Anthropic is a pioneering AI safety and research organization based in San Francisco, committed to developing reliable, interpretable, and steerable AI systems. With a strong focus on safety as a scientific discipline, Anthropic aims to generate insights on the opportunities and risks associated with AI developments. The company promotes collaboration with civil society, government, academia, and industry to foster a safer AI ecosystem for the world.

## Our Purpose
At Anthropic, we believe that AI will have a significant impact on the world. Our mission is to ensure that transformative AI benefits society and individuals alike. Our interdisciplinary team of researchers, engineers, policy experts, and operational leaders works collaboratively to tackle the challenges posed by advanced AI technologies.

## Products
### Claude
**Claude 3.5 Sonnet**: Our most advanced AI model designed to assist businesses in driving efficiency and creating new revenue streams. Claude serves as a virtual teammate, enhancing productivity through automation and intelligent collaboration.

### Enterprise Solutions
Anthropic provides enterprise-level solutions that seamlessly connect Claude to organizational knowledge, empowering teams to harness the full potential of AI in their workflows. Our offerings include tailored integrations that foster innovation and improve operational efficiencies across various functions like Engineering, Marketing, Sales, and Customer Support.

## Customers
Our diverse clientele consists of businesses, nonprofits, and civil society organizations that leverage Claude's capabilities to enhance their operations, creative processes, and overall productivity. Customers have reported significant improvements in efficiency, such as achieving project outcomes in markedly reduced timeframes.

## Team and Culture
At Anthropic, we uphold a culture of unusually high trust, emotional maturity, and intellectual openness. We operate as one cohesive team, valuing collaboration across disciplines to enrich our work. Our values reflect our commitment to our mission and our drive to ensure that our systems help society flourish.

### Company Values
1. **Here for the Mission**: We are dedicated to building AI systems that prioritize safety and human benefit.
2. **Unusually High Trust**: Our work environment fosters honesty and constructive disagreement.
3. **One Big Team**: Collaboration is central; input from all levels shapes our organizational goals.
4. **Do the Simple Thing That Works**: We embrace pragmatic, evidence-based approaches in our processes.

## Careers at Anthropic
Join us in our mission to make AI safe for the future. At Anthropic, we pride ourselves on hiring individuals from diverse backgrounds, ensuring a rich variety of perspectives in our team.

### What We Offer
- **Health & Wellness**: Comprehensive health benefits, inclusive fertility support, flexible paid time off, and generous mental health resources.
- **Compensation & Support**: Competitive salaries, equity packages, robust retirement plans, and additional perks like a wellness stipend and daily meals in our office.
- **Career Growth**: Opportunities to work on cutting-edge research and contribute to impactful AI technology, alongside a supportive and collaborative work environment.

### How to Apply
Our interview process is designed to minimize bias and highlight each candidate's strengths. It includes:
1. **Resume Submission**
2. **Exploratory Chat**
3. **Skills Assessment**
4. **Team Screening**
5. **Interview Panel**
6. **Final Checks and Offer**

We welcome talented individuals passionate about AI safety and innovation. Explore our open roles on our [Careers Page](https://www.anthropic.com/careers).

## Join Us
If you're eager to contribute to AI's future and want to explore innovative ways to ensure its safe development, we invite you to connect with us. Together, let's build AI that people can rely on.

**Website**: [Anthropic](https://www.anthropic.com)

**Social Media**: Follow us on [LinkedIn](https://www.linkedin.com/company/anthropic) and [Twitter](https://twitter.com/AnthropicAI) for the latest updates.

--- 

This brochure encapsulates Anthropic's mission, culture, product offerings, and career opportunities. We look forward to welcoming you to our team and the future of AI!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}


# Anthropic Brochure

---

## About Us

**Anthropic** is an AI safety and research company headquartered in San Francisco, dedicated to developing reliable, interpretable, and steerable AI systems. With the belief that AI will have a vast impact on the world, we focus on building systems that people can trust while generating research about the opportunities and risks of AI. Our mission is to ensure that transformative AI helps people and society flourish.

---

## Our Products

### Claude
Claude is our flagship AI assistant, designed to empower teams by enhancing productivity and collaboration. Users can utilize Claude for tasks such as:

- Drafting and iterating documents
- Writing and debugging code
- Creating marketing content
- Generating insightful reports

Claude is equipped to assist across various functions within organizations, ensuring that every team member has access to shared knowledge and expertise.

### API
Our API enables businesses to integrate Claude’s capabilities into their existing systems, driving efficiency and creating new revenue streams.

---

## Company Culture

At Anthropic, we emphasize a culture of **collaboration**, **trust**, and **intellectual openness**. Our values include:

1. **Here for the Mission**: Focused on ensuring that AI systems are safe and beneficial for society.
2. **Unusually High Trust**: We prioritize honesty and the assumption of good faith in all interactions.
3. **One Big Team**: Promoting collaboration across all teams, recognizing that every member contributes to the organization's shared goals.
4. **Pragmatic Approach**: We celebrate simplistic solutions to problems and rely on empirical evidence to guide our practices.

---

## Our Team

Anthropic's diverse team consists of researchers, engineers, policy experts, and operational leaders, all bringing different expertise to our mission. We foster an interdisciplinary approach, combining knowledge from various domains such as machine learning, physics, and public policy. Our leadership encourages input from all levels, ensuring that every voice contributes to the company’s strategic direction.

---

## Careers at Anthropic

We are always on the lookout for talented individuals who are passionate about AI safety and innovation. Our hiring process is designed to identify thoughtful candidates from varied backgrounds, with a commitment to reduce biases. 

### What We Offer
- **Health & Wellness**: Comprehensive healthcare, flexible paid time off, and generous mental health support.
- **Competitive Compensation**: Attractive salary, equity packages, and additional benefits like stipends for wellness and education.
- **Growth Opportunities**: As a public benefit corporation, we're invested in the long-term benefits of our employees and the communities we serve.

### Join Us
Check our [Careers page](https://anthropic.com/careers) for open roles and more information on how to apply.

---

## Customers

Our products benefit businesses, nonprofits, and civil society organizations globally. Customer success stories highlight tangible improvements in productivity and efficiency, thanks to Claude’s capabilities. 

---

## Follow Along

Stay updated on our latest innovations and developments by following us on social media:

- [Twitter](https://twitter.com/anthropic)
- [LinkedIn](https://linkedin.com/company/anthropic)
- [YouTube](https://youtube.com/@anthropic)

---

For more information, please visit our website at [Anthropic](https://anthropic.com) or reach out via our support channels.

**Anthropic** – *Building reliable, interpretable, and safe AI systems for a better future.*

In [23]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## About Us
**Hugging Face** is an innovative AI community dedicated to building the future through collaborative machine learning. Our platform serves as the home for researchers, developers, and organizations to create, discover, and utilize state-of-the-art models and datasets. With over 400,000 models and 100,000 datasets available, we foster a collaborative environment that empowers all participants in the machine learning ecosystem.

## Our Offerings
- **Models**: Access to a vast library of machine learning models across various modalities, including text, images, videos, and more.
- **Datasets**: Comprehensive datasets to facilitate training and evaluation across multiple domains.
- **Spaces**: Host and share interactive machine learning applications easily.
- **Enterprise Solutions**: Tailored offerings that focus on security, support, and features for large organizations.

## Customers
We proudly serve more than **50,000 organizations**, including prominent companies like:
- Amazon Web Services
- Google
- Microsoft
- Intel
- Grammarly

Our community continues to grow, making it a thriving hub for AI-driven companies.

## Company Culture
At Hugging Face, we prioritize community collaboration and open-source contributions. Our culture is built on the principles of transparency, inclusivity, and innovation, which are vital for advancing machine learning technology. We encourage our team members to engage in knowledge-sharing and continuous learning.

## Careers
We are constantly looking for talented individuals to join our team. Opportunities range from research and development roles to positions in sales, marketing, and operations. If you're interested in being part of a forward-thinking AI organization, explore our current openings on our [careers page](#).

## Join Us
Explore the exciting possibilities that Hugging Face has to offer! Whether you’re looking to collaborate on innovative machine learning models, deploy applications, or join our vibrant community, there's a place for you here.

**Get Started Today!**
- Visit us at [Hugging Face](https://huggingface.co/)
- Connect with our community on [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/huggingface/)

Together, let’s build the future of AI! 🌟

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>